In [1]:
# IMPORT STATEMENTS
import re
import numpy as np
from matplotlib import pyplot as plt
get_ipython().magic('matplotlib inline')
import pandas as pd
from collections import Counter
import os
import sys
from scipy.stats import fisher_exact, ttest_ind
import StepMiner as smn
import HegemonUtil as hu
acolor = ["#00CC00", "#D8A03D","#EC008C",
          'cyan', "#B741DC", "#808285",
          'blue', 'black', 'green', 'red',
          'orange', 'brown', 'pink', 'purple']

try:
    reload  # Python 2.7
except NameError:
    try:
        from importlib import reload  # Python 3.4+
    except ImportError:
        from imp import reload  # Python 3.0 - 3.3

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import bone
reload(bone)
class MacAnalysis(bone.MacAnalysis):

    def __init__(self):
        bone.MacAnalysis.__init__(self)
   
    def getGSE14580(self, tn=1):
        self.prepareData("PLP53")
        atype = self.h.getSurvName('c response to infliximab')
        atypes = ['R', 'NR']
        ahash = {'Yes':0, 'No':1}
        self.initData(atype, atypes, ahash)
        
    def getARIJS_1(self, tn=1):
        self.prepareData("CD2","/home/saptarshi.sinha/Hegemon/explore.conf")
        atype = self.h.getSurvName('c response to infliximab (ch1)')
        atypes = ['Yes_Before','No_Before']
        ahash = {'CD_Yes_Before first infliximab treatment':0,'CD_No_Before first infliximab treatment':1,'UC_Yes_Before first infliximab treatment':0,'UC_No_Before first infliximab treatment':1}
        self.initData(atype, atypes, ahash)   
        
    def getVanderGoten(self, tn=1):
        self.prepareData("PLP25")
        activity = self.h.getSurvName("c disease activity")
        atype = self.h.getSurvName("c disease")
        atypes = ['Control', 'UC', 'CD', 'I', 'A', 'NA']
        ahash = {'control':0,
                'active':4, 'inactive':3, 'not applicable': 5}
        if (tn == 2):
            atypes = ['I', 'A']
            ahash = {'active':1, 'inactive':0}
            atype = activity
        self.initData(atype, atypes, ahash) 
        
    def getVanhove(self, dtype=0, tn=1):
        self.prepareData("PLP23")
        activity = self.h.getSurvName("c disease activity")
        atype = self.h.getSurvName("c disease")
        atypes = ['Control', 'UC', 'CD']
        ahash = {'ulcerative colitis':1, "Crohn's disease":2, 'control':0}
        if (tn == 2):
            atypes = ['I', 'A']
            ahash = {'active':1, 'inactive':0}
            atype = activity
        self.initData(atype, atypes, ahash)        
    
def plotViolinBar(ana, desc=None):
    fig = plt.figure(figsize=(5,5), dpi=100)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    ax1 = plt.subplot2grid((4, 1), (0, 0))
    ax2 = plt.subplot2grid((4, 1), (1, 0), rowspan=3)
    params = {'spaceAnn': len(ana.order)/len(ana.atypes), 'tAnn': 1, 'widthAnn':1,
              'genes': [], 'ax': ax1, 'acolor': acolor}
    ax = ana.printTitleBar(params)
    res = ana.getROCAUC()
    ax.text(len(ana.cval[0]), 4, res)
    if desc is not None:
        ax.text(-1, 2, desc, horizontalalignment='right',
                    verticalalignment='center')
    params = {'spaceAnn': len(ana.order)/len(ana.atypes), 'tAnn': 1, 'widthAnn':1,
            'genes': [], 'ax': ax2, 'acolor': acolor, 'vert': 0}
    ax = ana.printViolin(None, params)
#    return fig

def plotDensityBar(ana, desc=None):
    fig = plt.figure(figsize=(4,4), dpi=100)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    ax1 = plt.subplot2grid((4, 1), (0, 0))
    ax2 = plt.subplot2grid((4, 1), (1, 0), rowspan=3)
    params = {'spaceAnn': len(ana.order)/len(ana.atypes), 'tAnn': 1, 'widthAnn':1,
              'genes': [], 'ax': ax1, 'acolor': acolor}
    ax = ana.printTitleBar(params)
    res = ana.getMetrics(ana.cval[0])
    ax.text(len(ana.cval[0]), 4, ",".join(res))
    if desc is not None:
        ax.text(-1, 2, desc, horizontalalignment='right',
                    verticalalignment='center')
    ax = ana.densityPlot(ax2, acolor)
    return fig

def processData(ana, l1, wt1, desc=None, violin=1):
    ana.orderData(l1, wt1)
    if (violin == 1):
        return plotViolinBar(ana, desc)
def processDataDf(ana, l1, wt1, desc=None):
    fig = plt.figure(figsize=(4,4), dpi=100)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    ax1 = plt.subplot2grid((4, 1), (0, 0))
    ax2 = plt.subplot2grid((4, 1), (1, 0), rowspan=3)

    c_dict, fpr, tpr, roc_auc = bone.processGeneGroupsDf(ana, l1, wt1)
    params = {'spaceAnn': len(ana.order)/len(ana.atypes), 'tAnn': 1, 'widthAnn':1,
              'genes': [], 'ax': ax1, 'acolor': acolor}
    ax = ana.printTitleBar(params)
    res = ana.getROCAUC()
    ax.text(len(ana.cval[0]), 4, res)
    if desc is not None:
        ax.text(-1, 2, desc, horizontalalignment='right',
                    verticalalignment='center')
    params = {'spaceAnn': len(ana.order)/len(ana.atypes), 'tAnn': 1, 'widthAnn':1,
            'genes': [], 'ax': ax2, 'acolor': acolor, 'vert': 0}
    ax = ana.printViolin(None, params)
    return fig

def getOrder(ana, l1):
    from scipy.stats import fisher_exact, ttest_ind
    res = []
    for s in l1:
        for gn in s:
            id1 = ana.h.getBestID(ana.h.getIDs(gn).keys())
            if id1 is None:
                continue
            e = ana.h.getExprData(id1)
            v1 = np.array([float(e[i]) if e[i] != "" else 0 for i in ana.state[0]])
            v2 = np.array([float(e[i]) if e[i] != "" else 0 for i in ana.state[1]])
            t, p = ttest_ind(v1,v2,equal_var=False)
            res += [[id1, ana.h.getSimpleName(id1),
                   t, p, np.mean(v1)-np.mean(v2)]]
    return pd.DataFrame(res, columns=['ProbeID', 'Name', 'T', 'p', 'Diff'])
def plotVolcano(ana, genes, cfile, ylim=[0, 10.5], xlim=[-6, 6]):
    df = processGenes(ana.h, [ana.state[0], ana.state[1]], genes)
    df['Size'] = 10
    fig,ax = plt.subplots(figsize=(6,4), dpi=100)
    crcdf = pd.read_csv(cfile)
    crcdf['logp'] = -np.log10(crcdf['pvalue'])
    ax = sns.scatterplot('log2FoldChange', 'logp', size=0.1, color='0.8',
                         edgecolor="none", data=crcdf)
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    ax.legend().set_visible(False)
    import io
    import base64
    buffer = io.BytesIO()
    fig.savefig(buffer, format='jpg')
    buffer.seek(0)
    volcano = base64.b64encode(buffer.read())
    from PIL import Image, ImageDraw
    buffer.seek(0)
    img = Image.open(buffer)
    x = list(ax.bbox.bounds)
    x[2] = x[2] + x[0]
    x[3] = x[3] + x[1] - 2
    x[1] = x[1] - 2
    img = img.crop(x)

    fig,ax = plt.subplots(figsize=(6,4), dpi=100)
    ax = sns.scatterplot('Diff', 'logp', hue='Diff', palette='vlag',
                         data=df, size='Size', size_norm=(0, 10), 
                         edgecolor="none", zorder=2, ax=ax)
    ax.legend().set_visible(False)
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    for i in df.index:
        ax.text(df['Diff'][i]+.02, df['logp'][i], str(df['Name'][i]))
    ax.imshow(img,
              aspect = ax.get_aspect(),
              extent = ax.get_xlim() + ax.get_ylim(),
              zorder = 1) #put the map under the heatmap
    return ax

def savePList(ofile, ana, l1):
    df = getOrder(ana, l1)
    df1 = df.sort_values(by=['T'], ascending=True)
    bone.saveList(ofile, df1['Name'])


def getSource(self):
    if 'source' in self.hash:
        return self.hash['source']
    print(f"source missing in hash: {self.hash}")
    return 'unknown_source'
    

Agonist

In [3]:
import bone
reload(bone)
db = hu.Database("explore.conf")
dbid = "PLP7"
h = hu.Hegemon(db.getDataset(dbid))
h.init()
h.initPlatform()
h.initSurv()

f1 = open("34.txt","r")
B=[]
for line in f1:
    for word in line.split():
        B.append(word)
f1.close()   


f3 = open("target_gene.txt","r")
C=[]
for line in f3:
    for word in line.split():
        C.append(word)
f3.close()

R=[]
f2 = open('S&P_result_agonist.txt','w')
f3 = open('result_agonist.txt','w')
f2.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\n'%('', 'ERMP1', 'SH3BGRL2', 'SLC35B3', 'BDH1', 'LIMA1', 'HSD11B2', 'KIF16B', 'PRELID2', 'TCEA3', 'MPND', 'COQ9', 'TMEM177', 'PGAP3', 'UQCRC1', 'COX5A', 'TST', 'MDH2', 'COX4I1', 'PLEKHJ1', 'GOT1', 'PPFIA3', 'SULT1A1', 'SPR', 'PKIB', 'FAM162A', 'GRAMD4', 'PAPSS2', 'CMTM4', 'BLOC1S1', 'THAP4', 'AKAP1', 'ZDHHC23', 'DTX4', 'COX10'))
ASc=[]
for i in C:
    f2.write('%s\t'%(i))
    #print (i)
    Aset=[]
    for j in B:
        if h.compareIds(j,i) ==0:
            f2.write('%s\t'%('NA'))
        else:
            counts, e, s, p = h.getBooleanRelation(j, i)
            if s[1]>1:
                #f2.write('%s\t'%("Antagonist"))
                thr_index = 0.25 * ((0.3/(s[1] + 1.0)) + (0.3/(s[2] + 1.0)) + p[1] + p[2])
                f2.write('%s\t'%(thr_index))
                if thr_index < 0.075:
                    ASc.append(j)
                    Aset.append(j)
            elif s[1]<=1:
                #f2.write('%s\t'%("Agonist"))
#                thr_index = 0.25 * ((0.3/(s[1] + 1.0)) + (0.3/(s[2] + 1.0)) + p[1] + p[2])                
                f2.write('%s\t'%('1'))  
#            thr_index=0.25 * ((0.3/(s[0] + 1.0)) + (0.3/(s[3] + 1.0)) + p[0] + p[3])
#            f2.write('%s\t'%(thr_index))
    f2.write('\n')
    print (i,':', ', '.join(Aset))
    print ('FRD score of',i,':',len(set(Aset)),(len(set(Aset))/34))
    f3.write('%s\t%s\n'%(i,(len(set(Aset))/34)))
    print ("************************")
f2.close()  
f3.close()
U=set(ASc)
print ("Agglomerative score: ",len(U),float(len(U)/34)) 

PRKAB1 : SH3BGRL2, SLC35B3, BDH1, HSD11B2, PRELID2, TCEA3, MPND, COQ9, TMEM177, PGAP3, UQCRC1, COX5A, TST, MDH2, COX4I1, PLEKHJ1, GOT1, PPFIA3, SULT1A1, SPR, PKIB, FAM162A, GRAMD4, PAPSS2, CMTM4, BLOC1S1, THAP4, AKAP1, ZDHHC23, DTX4, COX10
FRD score of PRKAB1 : 31 0.9117647058823529
************************
Agglomerative score:  31 0.9117647058823529


Antagonist

In [4]:
import bone
reload(bone)
db = hu.Database("explore.conf")
dbid = "PLP7"
h = hu.Hegemon(db.getDataset(dbid))
h.init()
h.initPlatform()
h.initSurv()

f1 = open("34.txt","r")
B=[]
for line in f1:
    for word in line.split():
        B.append(word)
f1.close()   


f3 = open("target_gene.txt","r")
C=[]
for line in f3:
    for word in line.split():
        C.append(word)
f3.close()

R=[]
f2 = open('S&P_result_antagonist.txt','w')
f3 = open('result_antagonist.txt','w')
f2.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\n'%('', 'ERMP1', 'SH3BGRL2', 'SLC35B3', 'BDH1', 'LIMA1', 'HSD11B2', 'KIF16B', 'PRELID2', 'TCEA3', 'MPND', 'COQ9', 'TMEM177', 'PGAP3', 'UQCRC1', 'COX5A', 'TST', 'MDH2', 'COX4I1', 'PLEKHJ1', 'GOT1', 'PPFIA3', 'SULT1A1', 'SPR', 'PKIB', 'FAM162A', 'GRAMD4', 'PAPSS2', 'CMTM4', 'BLOC1S1', 'THAP4', 'AKAP1', 'ZDHHC23', 'DTX4', 'COX10'))
ASc=[]
for i in C:
    f2.write('%s\t'%(i))
    #print (i)
    Aset=[]
    for j in B:
        if h.compareIds(j,i) ==0:
            f2.write('%s\t'%('NA'))
        else:
            counts, e, s, p = h.getBooleanRelation(j, i)
            if s[0]>1:
                #f2.write('%s\t'%("Antagonist"))
                thr_index = 0.25 * ((0.3/(s[0] + 1.0)) + (0.3/(s[3] + 1.0)) + p[0] + p[3])
                f2.write('%s\t'%(thr_index))
                if thr_index < 0.075:
                    ASc.append(j)
                    Aset.append(j)
            elif s[0]<=1:
                #f2.write('%s\t'%("Agonist"))
#                thr_index = 0.25 * ((0.3/(s[1] + 1.0)) + (0.3/(s[2] + 1.0)) + p[1] + p[2])                
                f2.write('%s\t'%('1'))  
#            thr_index=0.25 * ((0.3/(s[0] + 1.0)) + (0.3/(s[3] + 1.0)) + p[0] + p[3])
#            f2.write('%s\t'%(thr_index))
    f2.write('\n')
    print (i,':', ', '.join(Aset))
    print ('FRD score of',i,':',len(set(Aset)),(len(set(Aset))/34))
    f3.write('%s\t%s\n'%(i,(len(set(Aset))/34)))
    print ("************************")
f2.close()    
f3.close()    
U=set(ASc)
print ("Agglomerative score: ",len(U),float(len(U)/34)) 

PRKAB1 : 
FRD score of PRKAB1 : 0 0.0
************************
Agglomerative score:  0 0.0
